# Training of the Extended Mask2Former

In [1]:
# Import Libraries
import sys, os
module_path = os.path.abspath(os.path.join('..'))
import torch
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from src.data_preprocessing import CocoDataset
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Training parameters
EPOCHS = 10
BATCH_SIZE = 16
LEARNING_RATE = 1e-3

ModuleNotFoundError: No module named 'data_preprocessing'

### Set up GPU growth

In [ ]:
# Set device we are going to load the model and the data
device = 'mps' if torch.backends.mps.is_available() else "cpu"

### Data Preprocessing 

In [ ]:
# Import the data from the COCO dataset
_dir = 'path/to/coco/images'  # Update this path
annotation_file = 'path/to/annotations/instances_train2017.json'  # Update this path


# Define the Transformations
transform = transforms.Compose([transforms.Resize((600, 600)), transforms.ToTensor()])


coco_dataset = COCODataset(root_dir=root_dir, annotation_file=annotation_file, transform=transform)
data_loader = DataLoader(coco_dataset, batch_size=4, shuffle=True, num_workers=4)


In [ ]:

# Define your model, loss function, and optimizer
model = ExtendedMaskTransformer(...)  # Initialize your model
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = ...  # Define your loss function

# Training loop (simplified)
for epoch in range(EPOCHS):
    for images, targets in data_loader:
        images = images.to(device)
        targets = targets.to(device)
        
        # Forward pass
        outputs = model(images)
        
        # Compute loss
        loss = loss_fn(outputs, targets)
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Note: This is a very simplified training loop. You'll need to adjust it based on your model's specific inputs and outputs.
